# Configure

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Packages
import os
from os.path import join
from pathlib import Path
import yaml
from yaml.loader import SafeLoader
import pandas as pd
import geopandas as gpd
import numpy as np

import unsafe.download as undown
import unsafe.files as unfile
import unsafe.unzip as ununzip
import unsafe.exp as unexp
import unsafe.ddfs as unddf
import unsafe.ensemble as unens

In [ ]:
# Name the fips, statefips, stateabbr, and nation that
# we are using for this analysis
# We pass these in as a list even though the framework currently
# processes a single county so that it can facilitate that
# expansion in the future
# TODO - could make sense to define these in the future
# in json or other formats instead of as input in code
fips_args = {
    'FIPS': ['42101'], 
    'STATEFIPS': ['42'],
    'STATEABBR': ['PA'],
    'NATION': ['US']
}
FIPS = fips_args['FIPS'][0]
NATION = fips_args['NATION'][0]


In [ ]:
# We need to pass in a config file that sets up
# constants and the structure for downlading data
# For the directory structure of our case study, 
# we use the following 
ABS_DIR = Path().absolute().parents[0]

CONFIG_FILEP = join(ABS_DIR, 'config', 'config.yaml')
# Open the config file and load
with open(CONFIG_FILEP) as f:
    CONFIG = yaml.load(f, Loader=SafeLoader)

# Wildcards for urls
URL_WILDCARDS = CONFIG['url_wildcards']

# Get the file extensions for api endpoints
API_EXT = CONFIG['api_ext']

# Get the CRS constants
NSI_CRS = CONFIG['nsi_crs']

# Dictionary of ref_names
REF_NAMES_DICT = CONFIG['ref_names']

# Dictionary of ref_id_names
REF_ID_NAMES_DICT = CONFIG['ref_id_names']

# Coefficient of variation
# for structure values
COEF_VARIATION = CONFIG['coef_var']

# First floor elevation dictionary
FFE_DICT = CONFIG['ffe_dict']

# Number of states of the world
N_SOW = CONFIG['sows']

# Get hazard model variables
# Get Return Period list
RET_PERS = CONFIG['RPs']
HAZ_FILEN = CONFIG['haz_filename']
# Get CRS for depth grids
HAZ_CRS = CONFIG['haz_crs']

# Get the files we need downloaded
DOWNLOAD = pd.json_normalize(CONFIG['download'], sep='_').T

# We can also specify the filepath to the
# raw data directory
FR = join(ABS_DIR, "data", "raw")

# And external - where our hazard data should be
FE = join(FR, "external")

# Set up interim and results directories as well
# We already use "FR" for raw, we use "FO" 
# because you can also think of results
# as output
FI = join(ABS_DIR, "data", "interim")
FO = join(ABS_DIR, "data", "results")

# "Raw" data directories for exposure, vulnerability (vuln) and
# administrative reference files
EXP_DIR_R = join(FR, "exp")
VULN_DIR_R = join(FR, "vuln")
REF_DIR_R = join(FR, "ref")
# Haz is for depth grids
HAZ_DIR_R = join(FE, "haz")
# Pol is for NFHL
POL_DIR_R = join(FR, "pol")

# Unzip directory 
UNZIP_DIR = join(FR, "unzipped")

# We want to process unzipped data and move it
# to the interim directory where we keep
# processed data
# Get the filepaths for unzipped data
# We unzipped the depth grids (haz) and 
# ddfs (vuln) into the "external"/ subdirectory
HAZ_DIR_UZ = join(UNZIP_DIR, "external", "haz")
POL_DIR_UZ = join(UNZIP_DIR, "pol")
REF_DIR_UZ = join(UNZIP_DIR, "ref")
VULN_DIR_UZ = join(UNZIP_DIR, "external", "vuln")

# "Interim" data directories
EXP_DIR_I = join(FI, "exp")
VULN_DIR_I = join(FI, "vuln")
REF_DIR_I = join(FI, "ref")
# Haz is for depth grids
HAZ_DIR_I = join(FI, "haz")
# Pol is for NFHL
POL_DIR_I = join(FI, "pol")

# Download (and unzip) data

In [ ]:
wcard_dict = {x: fips_args[x[1:-1]][0] for x in URL_WILDCARDS}
undown.download_raw(DOWNLOAD, wcard_dict,
                    FR, API_EXT)

In [ ]:
ununzip.unzip_raw(FR, UNZIP_DIR)

# Prepare data for ensemble

## Exposure

In [ ]:
nsi_gdf = unexp.get_nsi_geo(FIPS, NSI_CRS, EXP_DIR_R)

# Set the values that we pass into the get_struct_subset
# function. In this case, occtype==RES1 and num_story <= 2
occtype_list=['RES1-1SNB', 'RES1-2SNB', 'RES1-1SWB', 'RES1-2SWB']
sub_string = 'occtype.isin(@occtype_list) and num_story <= 2'
nsi_sub = unexp.get_struct_subset(nsi_gdf,
                                  filter=sub_string,
                                  occtype_list=occtype_list)

# For this case study, let us save some memory and just
# write out the single family houses 
EXP_OUT_FILEP = join(EXP_DIR_I, FIPS, 'nsi_sf.gpkg')
unfile.prepare_saving(EXP_OUT_FILEP)
nsi_sub.to_file(EXP_OUT_FILEP, driver='GPKG')

## Reference

In [ ]:
# We are going to clip reference data to a clip file that
# represents our study boundaries. In this case, it's the county
# of Philadelphia, so we will prepare that as our clip file
county_filep = join(REF_DIR_UZ, NATION, 'county', 'tl_2022_us_county.shp')
county_gdf = gpd.read_file(county_filep)
clip_gdf = county_gdf[county_gdf[REF_ID_NAMES_DICT['county']] == FIPS]

# clip_ref_files will go through all unzipped ref files,
# clip them in the clip file geometry, and write them
# We pass in arguments we defined in the configuration step
# to tell the function where the data can be found (REF_DIR_UZ),
# where is is going (REF_DIR_I), and how to update id names
# (REF_NAMES_DICT). 
# Subset of ref downloads
ref_downloads = DOWNLOAD[DOWNLOAD.index.str.contains('_ref_')]
# Clip ref to catchment
# Use FIPS as clip_str since the catchment is in a county
# and this is eaiser for directory management
unexp.clip_ref_files(clip_gdf, FIPS, fips_args, ref_downloads,
                     wcard_dict, REF_DIR_UZ, REF_DIR_I)

## Physical vulnerability

In [ ]:
unddf.process_naccs(VULN_DIR_UZ, VULN_DIR_I)

# .3 was used in Zarekarizi et al. 2020
# https://www.nature.com/articles/s41467-020-19188-9
# and we are going to use that for this case study
UNIF_UNC = .3
unddf.process_hazus(VULN_DIR_UZ, VULN_DIR_I, unif_unc=UNIF_UNC)

## Social vulnerability

In [ ]:
# Process national social vulnerability data
# Tell the function which datasets we want processed
# In this case study, we will use cejst
# which is still available nationally

sovi_list = ['cejst']
unexp.process_national_sovi(sovi_list, FIPS,
                            VULN_DIR_R, REF_DIR_I, VULN_DIR_I)

## Policy

In [ ]:
# We need NFHL for the ensemble and visualizations
unexp.process_nfhl(FIPS,
                   POL_DIR_UZ,
                   POL_DIR_I,
                   "Philadelphia_NFHL_42_2023.shp")

## Link flood zones and references to structures

In [ ]:
# Link flood zones
# We checked for issues like overlapping flood zones
# resulting in NSI structures in multiple polygons
# and did not find any. That's good, but chances
# are there will be counties where this happens
# and we will need code to handle these consistently for other
# case studies
nfhl_filep = join(POL_DIR_I, FIPS, 'fld_zones.gpkg')
nfhl = gpd.read_file(nfhl_filep)
keep_cols = ['fld_zone', 'fld_ar_id', 'static_bfe']

# This function is designed to work in general
# for any spatial-based data you want to link
# to structures. We demonstrate it with the
# nfhl data. 
unexp.get_spatial_var(nsi_sub,
                      nfhl,
                      'fz',
                      FIPS,
                      EXP_DIR_I,
                      keep_cols)

# Link references
# This will do spatial joins for structures within
# all the reference spatial files (besides county)
# and output a file of fd_id (these are unique strucutre ids)
# linked to all of the reference ids
nsi_refs = unexp.get_ref_ids(nsi_sub.set_index('fd_id'), FIPS,
                             REF_ID_NAMES_DICT, REF_DIR_I, EXP_DIR_I)

nsi_ref_filep = join(EXP_DIR_I, FIPS, "nsi_ref.pqt")
unfile.prepare_saving(nsi_ref_filep)
nsi_refs.to_parquet(nsi_ref_filep)

## Hazard

In [ ]:
# Sample the inundation grids and write out the
# fd_id/depths dataframe
depth_df = unexp.get_inundations(nsi_sub,
                                 HAZ_CRS, RET_PERS,
                                 HAZ_DIR_UZ, HAZ_FILEN)

# Because we are processing design-event based flood scenarios,
# we can provide more helpful column names to prepare our loss
# and expected annual loss calculations. We will also
# write out our dataframe. 
ncol = [str(round(100/float(x.replace('_', '.')))) for x in depth_df.columns]
depth_df.columns = ncol

# Write out dataframe that links fd_id to depths
# with columns corresponding to ret_per (i.e. 500, 100, 50, 10)
# in our case study
nsi_depths_out = join(EXP_DIR_I, FIPS, 'nsi_depths.pqt')
unfile.prepare_saving(nsi_depths_out)
depth_df.reset_index().to_parquet(nsi_depths_out)

# Generate ensemble

In [ ]:
# Get a dataframe conducive for loss estimation
# This procedure is separate from preparing data for the ensemble
# so will just take the county code to load in and merge
# all the relevant data

run_config = {
    'ddfs': ['hazus', 'naccs'],
    'struct_list': ['ffe', 'val_struct', 'num_story', 'found_type'],
    'base_adj': False
}

base_df = nsi_sub.set_index('fd_id')
# We only want the RES1 classification from the NSI
# for damage function assignments
# (not their 1SWB, etc., type classifications)
base_df['occtype'] = base_df['occtype'].str[:4].copy()
# We can easily read in our census tract links from above,
# but we use this shortcut in the interest
# of keeping the tutorial more readable and 
# focused on functionality
base_df['tract_id'] = base_df['cbfips'].str[:11]
# For HAZUS DDFs, we need to link a property to its
# flood zone. Since Philadelphia is not subject to 
# wave action, we automatically use the A zone
# curves (as opposed to Z). For other
# case studies, if V zones are appropriate,
# you need to merge in the flood zone data
# we linked to parcels above
base_df['fz_ddf'] = 'A'

# We demonstrate the ensemble generation when all sources of uncertainty vary
ens_df_f = unens.get_loss_ensemble(base_df,
                                   depth_df,
                                   run_config,
                                   VULN_DIR_I)

In [ ]:
eals = {}
for ddf in ['hazus', 'naccs']:
    col_sub = [x for x in ens_df_f if ddf + '_loss' in x]
    loss_sub = ens_df_f[col_sub]
    # Update column name to remove reference to ddf
    loss_sub.columns = ['_'.join(x.split('_')[1:]) for x in col_sub]
    ret_per_ints = [int(x.split('_')[-1]) for x in col_sub]
    rp_list = sorted(ret_per_ints)
    eals[ddf] = unddf.get_eal(loss_sub, rp_list)
eals = pd.DataFrame.from_dict(eals)
eals.columns = [x + '_eal' for x in eals.columns]
ens_df_out = pd.concat([ens_df_f, eals], axis=1)

In [ ]:
# Save dataframes
base_out_filep = join(FO, 'base_df.pqt')
unfile.prepare_saving(base_out_filep)
base_df.to_parquet(base_out_filep)

ens_out_filep = join(FO, 'ensemble.pqt')
ens_df_out.to_parquet(ens_out_filep)

# Estimate benchmark losses

In [ ]:
# We also want benchmark estimates without uncertainty 
# which we can do with the full_df specified above
nounc_df = unens.benchmark_loss(base_df, depth_df, VULN_DIR_I, base_adj=False)
rp_list = sorted([int(x.split("_")[-1]) for x in nounc_df.columns])
eals = unddf.get_eal(nounc_df, rp_list)

nounc_df = pd.concat([nounc_df, pd.Series(eals, name='eal')], axis=1)

hazus_def_out_filep = join(FO, 'benchmark_loss.pqt')
unfile.prepare_saving(hazus_def_out_filep)
nounc_df.to_parquet(hazus_def_out_filep)

# Example plots

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import seaborn as sns

Read the loss estimates.

In [ ]:
nounc_df = pd.read_parquet(join(FO, 'benchmark_loss.pqt'))
ens_df_f = pd.read_parquet(join(FO, 'ensemble.pqt'))

Compare the no uncertainty estimate to the distributions from estimating damages under uncertainty with two different damage functions.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=300,
                       nrows=2,
                       sharex=True,
                       gridspec_kw={'hspace': 0},
                       height_ratios=[1,3])
# When we groupby 'sow_ind' we are looking across ensemble members
temp = ens_df_f.groupby(['sow_ind'])[['naccs_eal', 'hazus_eal']].sum()/1e6
temp['naccs_eal'].hist(bins=50, color='blue', alpha=.5, label='NACCS DDFs')
temp['hazus_eal'].hist(bins=50, color='orange', alpha=.5, label='HAZUS DDFs')
ax[1].axvline(nounc_df['eal'].sum()/1e6, color='red', label='No Uncertainty')
ax[0].axvline(nounc_df['eal'].sum()/1e6, color='red', label='No Uncertainty')
ax[1].grid(False)
ax[1].set_xlabel('Total Expected Annual Loss ($ Millions)', size=14)
ax[1].set_ylabel('Number of Ensemble Members', size=14)
ax[1].tick_params(labelsize=12)

temp.columns = ['NACCS', 'HAZUS']
temp_box = temp.melt(value_name='eal',
                     var_name='DDF Type')
sns.boxplot(ax=ax[0],
            data=temp_box,
            x='eal',
            hue='DDF Type',
            legend=False,
            showmeans=True,
            meanprops={'markerfacecolor': 'firebrick',
                        'markeredgecolor': 'black',
                        'marker': 'D'})
ax[0].axis('off')


# Easier to do a custom legend
legend_elements = [Patch(facecolor=sns.color_palette("tab10")[1],
                         label='HAZUS DDFs'),
                    Patch(facecolor=sns.color_palette("tab10")[0],
                         label='NACCS DDFs'),
                   Line2D([0], [0], color='r', lw=2, label='No Uncertainty'),
                   Line2D([0], [0], marker='D', markerfacecolor='firebrick',
                          label='Ensemble Mean',
                          ls='',
                          markeredgecolor='black', markersize=8),]


ax[1].legend(handles=legend_elements,
             loc='upper right',
             fontsize='x-large',
             frameon=False)

Can do the same for a single house.

In [ ]:
# Let's plot the above for a house with some zero EAL 
# to demonstrate some ways to evaluate ensemble results

check_prop_id = ens_df_f[ens_df_f['hazus_eal'] == 0]['fd_id'].unique()
check_prop = ens_df_f[ens_df_f['fd_id'].isin(check_prop_id)]

max_eal_id = check_prop.groupby('fd_id')['hazus_eal'].mean().idxmax()

max_unc = ens_df_f[ens_df_f['fd_id'] == max_eal_id]
max_nounc = nounc_df.loc[max_eal_id]

fig, ax = plt.subplots(figsize=(10, 6), dpi=300,
                       nrows=2,
                       sharex=True,
                       gridspec_kw={'hspace': 0},
                       height_ratios=[1,3])

temp = max_unc.groupby(['sow_ind'])[['naccs_eal', 'hazus_eal']].sum()/1e3

temp['naccs_eal'].hist(bins=100, color='blue', alpha=.5, label='NACCS DDFs')
temp['hazus_eal'].hist(bins=100, color='orange', alpha=.5, label='HAZUS DDFs')
ax[1].axvline(max_nounc['eal'].sum()/1e3, color='red', label='No Uncertainty')
ax[0].axvline(max_nounc['eal'].sum()/1e3, color='red', label='No Uncertainty')
ax[1].grid(False)
ax[1].set_xlabel('Expected Annual Loss ($ Thousands)', size=14)
ax[1].set_ylabel('Number of Ensemble Members', size=14)
ax[1].tick_params(labelsize=12)

temp.columns = ['NACCS', 'HAZUS']
temp_box = temp.melt(value_name='eal',
                     var_name='DDF Type')
sns.boxplot(ax=ax[0],
            data=temp_box,
            x='eal',
            hue='DDF Type',
            legend=False,
            showmeans=True,
            meanprops={'markerfacecolor': 'firebrick',
                        'markeredgecolor': 'black',
                        'marker': 'D'})
ax[0].axis('off')



# Easier to do a custom legend
legend_elements = [Patch(facecolor=sns.color_palette("tab10")[1],
                         label='HAZUS DDFs'),
                    Patch(facecolor=sns.color_palette("tab10")[0],
                         label='NACCS DDFs'),
                   Line2D([0], [0], color='r', lw=2, label='No Uncertainty'),
                   Line2D([0], [0], marker='D', markerfacecolor='firebrick',
                          label='Ensemble Mean',
                          ls='',
                          markeredgecolor='black', markersize=8),]


ax[1].legend(handles=legend_elements,
             loc='upper right',
             fontsize='x-large',
             frameon=False)

Interestingly, we see that there are some ensemble members with no (or little - hard to tell due to the grouping) expected annual loss, regardless of the DDF. In addition to this, there is evidence of two other modes in the distribution. We can look into the reasons for this. Let's check the realizations for foundation type and first-floor elevation. 

In [ ]:
# To start, let's look at when exp. annual loss
# is zero and non-zero when using hazus ddfs
max_unc['Expected Annual Loss'] = np.where(max_unc['hazus_eal'] > 0,
                                           '> 0',
                                           '= 0')

# We can bin the ffe column to make it easier to plot
max_unc['ffe_bin'] = pd.cut(max_unc['ffe'], bins=[0, 1, 2, 3, 4])

# Let's plot the first floor elevation on the x-axis
# and let's plot foundation type in different
# subplots. We will plot count on the y-axis.
# We will do the bar colors based on whether
# EAL = 0 or > 0
sns.catplot(x='ffe_bin',
            row='fnd_type',
            data=max_unc,
            hue='Expected Annual Loss',
            kind='count')

We see that very few of the ensemble members have zero expected annual loss. Those that do have crawl space foundation and first-floor elevation of 4 feet. It makes sense that houses with higher first-floor elevation will have less damage, all else equal. Let's look at boxplots for each foundation type/first-floor elevation combo to get a better handle of which ensemble members are driving the multi-modal loss distribution we saw. 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=300,
                       nrows=2,
                       sharex=True,
                       gridspec_kw={'hspace': 0},
                       height_ratios=[1,3])

g= sns.boxplot(data=max_unc,
            x='ffe_bin', y='hazus_eal',
            hue='fnd_type',
            ax=ax[1])

# replace legend labels
new_labels = ['Slab', 'Basement', 'Crawl Space']
for t, l in zip(g.legend_.texts, new_labels):
    t.set_text(l)

g= sns.countplot(data=max_unc,
                 x='ffe_bin',
                 hue='fnd_type',
                 ax=ax[0],
                 legend=False)

The above diagnostic plot shows that slab and crawl space houses, with identifical first-floor elevation to basement houses, tend to have lower expected annual losses (with inundation equal). Since crawl space and slab houses are considered "no basement" they have different DDFs than basement houses, so this plot just reflects differences in the DDFs. We can see that crawl space houses at >= 2 feet, and some realizations of slab houses at 1 foot, account for the lower part of the bimodial histogram shown two plots ago. There is also a third mode, around $20,000 in the plot shown two figures ago. This appears to be driven by many of the realizations of slab and crawl space houses with 1 foot first-floor elevation, and the basement foundation realizations at 3 and 4 feet first-floor elevation. 

This set of diagnostics shows that even for the house with the highest expected annual loss, there is substantial uncertainty in expected annual loss estimation due to interacting uncertainty in features like foundation type and first-floor elevation. These are often not included in parcel datasets, and as highlighted in our technical documentation there is very little guidance on how accurate the National Structure Inventory records are for these features. 

These diagnostics also demonstrate that analysts may want to be careful in reporting expected annual losses alone, at least for single properties, since it can hide the bimodial and varying nature of expected annual loss estimates across a structure's ensemble members. 

A common use case is to plot property-level risk over space, and overlay social vulnerability data over that. Here, we will demonstrate how the reference and social vulnerability processing steps we implemented into our workflow make this easy. 

In [ ]:
# We processed CEJST and CDC SVI data
# We will demonstrate the use case described above
# with the CEJST data because it has an indicator variable
# for when a community is considered "disadvantaged"
# and we only return those census tracts,
# which makes it a simpler plot
cejst = gpd.read_file(join(VULN_DIR_I, 'social', FIPS, 'cejst.gpkg'))
# So, we will also need census tract boundaries in our study area
# to delineate areas that are not "disadvantaged"
tract_geo = gpd.read_file(join(REF_DIR_I, FIPS, "tract.gpkg"))
# We also wrote out a .gpkg file of the single family houses
nsi_geo = gpd.read_file(join(EXP_DIR_I, FIPS, "nsi_sf.gpkg"))

In [ ]:
# We can filter the single family houses to those which face
# risk. We will first get the average expected annual loss
# across ensemble members. We use the HAZUS DDFs here, but
# you can test this out with the NACCS ones if you would like
ens_agg = ens_df_f.groupby('fd_id')['hazus_eal'].mean().reset_index()

nsi_risk = nsi_geo.merge(ens_agg,
                         how='inner',
                         on='fd_id')


In [ ]:
from matplotlib.patches import Patch
from matplotlib import ticker

# Helps to zoom in on places where at-risk homes are
# These were chosen to focus on an area where there
# were many houses with high expected risk.
minx = -75.10
maxx = -75.065
miny = 40
maxy = 40.02

fig, ax = plt.subplots(figsize=(6, 4),
                       dpi=300)

# Reproject cejst to tract crs
cejst_plot = cejst.to_crs(tract_geo.crs)

# Plot the "disadvantaged" tracts
# in light blue
cejst_plot.plot(ax=ax,
                color='#18437f',
                alpha=.3,
                edgecolor='none')

# Plot census tract boundaries
tract_geo.plot(ax=ax,
               edgecolor='black',
               color='none',
               lw=.5)

# Plot the eal
plot_col = 'hazus_eal'
vmin_eal = 0
vmax_eal = nsi_risk[plot_col].max()
# Sort before plotting
nsi_plot = nsi_risk.sort_values(plot_col, ascending=True)
nsi_plot.plot(ax=ax,
              column=plot_col,
              cmap='Reds',
              s=1,
              vmin=vmin_eal,
              vmax=vmax_eal)

# Set axis off but keep outline
ax.tick_params(axis='both',
               which='both',
               bottom=False,
               left=False,
               labelbottom=False,
               labelleft=False)

# Set bounds
ax.set_xlim([minx, maxx])
ax.set_ylim([miny, maxy])

legend_elements = [Patch(facecolor='#18437f',
                         alpha=.3,
                         label='Disadvantaged\nCommunity'),]
        
ax.legend(handles=legend_elements,
                  loc='center',
                  fontsize='small',
                  bbox_to_anchor=(.85, .19),
                  frameon=False)

# Add continuous legends
cax = fig.add_axes([.21, 0.06, .61, 0.02])
sm = plt.cm.ScalarMappable(cmap='Reds',
                           norm=plt.Normalize(vmin=vmin_eal,
                                              vmax=vmax_eal))
sm._A = []
cbr = fig.colorbar(sm, cax=cax, orientation='horizontal')
cbr.ax.tick_params(labelsize=12, rotation=15) 
cbr.set_label("Average Risk Across Ensemble ($)", size=12)

Hopefully you have a sense for how unsafe standardizes risk _estimation_ under uncertainty so that you can focus on risk _assessment_ built on risk estimates. We will continue to add use-cases to our library of examples, and expand the functionality of unsafe for more occupancy types, damage functions, structure inventories, and more! Please consider contributing through development, testing, and requesting features. 